**Task**: Run error correction for 7-qb Steane code
* Build the Stean EC circuit.
* Run the circuit on SparseSim to track propagation of stabilizer group through circuit.
* Add noise, measure the syndrome and perform necessary correction.

In [1]:
import pecos as pc
import numpy as np
from bitstring import BitArray

**(a)** Run circuit, apply correction if necessary to produce a $|0_L\rangle$ state on the data qubits.

In [2]:
n_qbs = 10 # 7 data_qbs, 3 msmt_qbs

k1 = 0b0001111
k2 = 0b1010101
k3 = 0b0110011
stab_gens = [k1,k2,k3] # generators of stabilizer group
stabs = [0,k1,k2,k3,k1^k2,k2^k3,k1^k3,k1^k2^k3]

plaquettes = [(3,4,5,6),(0,2,4,6),(1,2,5,6)] # correspond to stab set.
msmt_qbs = [7,8,9]
COR_TABLE = { 7: 3, 8: 0, 9: 1, 7+8: 4, 8+9: 2, 7+9: 5, 7+8+9: 6 }

# Plaquette measurements:
# 1. Init msmt qubits in plaquette basis (X or Z)
# 2. Connect CNOTS between data qbs and msmt qbs, s.t. plaquette basis errors can propagate from data qbs to msmt qbs.
# 3. Measure msmt qubits in plaquette basis.
# 4. Correct data qb error of OPPOSITE BASIS by syndrome (i.e. X-plaquette msmt finds & corrects Z-errors on data qbs)
def plaquette_qc(basis):
    qc = pc.circuits.QuantumCircuit()
    # qc.append('init |0>', {0,1,2,3,4,5,6})
    qc.append('init %s' % ('|0>' if basis == 'Z' else '|+>'), {7,8,9})
    for m_qb, plaquette in zip(msmt_qbs, plaquettes):
        for d_qb in plaquette:
            if basis == 'Z': qc.append('CNOT', {(d_qb, m_qb)})   # detect and correct X errors
            elif basis == 'X': qc.append('CNOT', {(m_qb, d_qb)}) # detect and correct Z errors
    qc.append('measure %s' % basis, {7,8,9}) # measure msmt qbs
    return qc

steane_x = plaquette_qc(basis='X')
steane_z = plaquette_qc(basis='Z')

# run plaquette msmts
circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
state = pc.simulators.SparseSim(n_qbs)
x, _ = circ_runner.run(state, steane_x)
z, _ = circ_runner.run(state, steane_z)

# apply corrective action
x_err_id = sum(list(*x.values()))
z_err_id = sum(list(*z.values()))
if x_err_id: 
    state.run_gate('Z', {COR_TABLE[x_err_id]}) # correct by opposite gate!
    print('X Syndrome', *x.values(), 'caused Z correction on qb.', COR_TABLE[x_err_id])
if z_err_id: 
    state.run_gate('X', {COR_TABLE[z_err_id]})
    print('Z Syndrome', *z.values(), 'caused X correction on qb.', COR_TABLE[z_err_id])

# Verify that the constructed state is |0_L>
qc = pc.circuits.QuantumCircuit()
qc.append('measure Z', {0,1,2,3,4,5,6})
msmt, _ = circ_runner.run(state, qc)

hamming2 = lambda a,b: bin(a^b).count('1')

# Msmt to bit string
msmt_bit_str = BitArray(bin='0000000')
for idx in list(*msmt.values()):
    msmt_bit_str[idx] = 1 # replace 0->1 at msmt index

# Hamming distance of measured state to each stab (as bitstrings)
dists = [hamming2(msmt_bit_str.uint, stab) for stab in stabs]
if min(dists) == 0:
    print("State",msmt_bit_str,"is +1 eigenstate of stabilizer", bin(stabs[dists.index(0)]), "with even parity, thus a |0_L> state.")

X Syndrome {8: 1} caused Z correction on qb. 0
State 0b0001111 is +1 eigenstate of stabilizer 0b1111 with even parity, thus a |0_L> state.


**(b)** Run the same circuit in the face of gate-wise noise. This time, repeat until faults occured that conspired to yield an error on the data qubits which we'd correct to a $|1_L\rangle$ state.

In [36]:
# %%timeit

# Create the noise model
class DepolarGen(pc.error_gens.parent_class_error_gen.ParentErrorGen):
    
    two_qubit_gates = {'CNOT', 'CZ', 'SWAP', 'G', 'MS', 'SqrtXX', 'RXX'}
    one_qubit_gates = {'I', 'X', 'Y', 'Z', 'Q', 'Qd', 'R', 'Rd', 'S', 'Sd', 
                       'H', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H+z+x', 'H-z-x', 
                       'H+y-z', 'H-y-z', 'H-x+y', 'H-x-y', 'F1', 'F1d', 'F2', 
                       'F2d', 'F3', 'F3d', 'F4', 'F4d', 'RX', 'RY', 'RZ'}
    pauli_errors = ['X','Y','Z']
    
    def start(self, circuit, error_params, state):
        super().start(circuit, error_params)
        
    def generate_tick_errors(self, tick_circ, time, **params):
        if type(time) == tuple: tick_index = time[-1]
        else: tick_index = time
        
        before = pc.circuits.QuantumCircuit() # faults before tick
        after  = pc.circuits.QuantumCircuit() # faults after tick
        
        q0 = lambda locs, fault, r: [{fault: {l}} for l in locs if np.random.random() <= r]
        q1 = lambda locs, faults, r: [{np.random.choice(faults): {l}} for l in locs if np.random.random() <= r]
        q2 = lambda locs, faults, r: [{np.random.choice(faults): {l}} for ltup in locs  if np.random.random() <= r for l in ltup]
        
        qc_append = lambda qc, circuit_setup: [qc.append(tick) for tick in circuit_setup]
        
        for sym, locs, _ in tick_circ.circuit.items(tick_index):
            if sym in {'init |0>', 'init |1>'}:
                qc_append(after, q0(locs, 'X', self.error_params['r']))
            elif sym in {'init |+>', 'init |->', 'init |+i>', 'init |-i>'}:
                qc_append(after, q0(locs, 'Z', self.error_params['r']))
            elif sym in {'measure Z'}:
                qc_append(before, q0(locs, 'X', self.error_params['q']))
            elif sym in {'measure X', 'measure Y'}:
                qc_append(before, q0(locs, 'Z', self.error_params['q']))
            elif sym in self.one_qubit_gates:
                qc_append(after, q1(locs, self.pauli_errors, self.error_params['p1']))
            elif sym in self.two_qubit_gates:
                qc_append(after, q2(locs, self.pauli_errors + ['I'], self.error_params['p2']))
            elif symbol == 'wait_initial' or symbol == 'wait_final':
                pass
            else:
                raise Exception("This error model doesn't handle gate: %s!" % symbol)
        self.error_circuits.add_circuits(time, before_faults=before, after_faults=after)
        return self.error_circuits

# Run circuit with error model
circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))

k1 = 0b0001111
k2 = 0b1010101
k3 = 0b0110011
stab_gens = [k1,k2,k3] # generators of stabilizer group
stabs = [0,k1,k2,k3,k1^k2,k2^k3,k1^k3,k1^k2^k3]
hamming2 = lambda a,b: bin(a^b).count('1')

min_dist = 2
cnt = 0
while min_dist > 1:
    
    # Run Plaquette measurements
    state = pc.simulators.SparseSim(n_qbs)
    a = 0.1
    e_params = {'q':a,'p1':a,'p2':a,'r':a}
    x, e_x = circ_runner.run(state, steane_x, error_gen=DepolarGen(), error_params=e_params)
    z, e_z = circ_runner.run(state, steane_z, error_gen=DepolarGen(), error_params=e_params)

    # Apply corrective action, if any
    x_err_id = sum(list(*x.values()))
    z_err_id = sum(list(*z.values()))
    if x_err_id: state.run_gate('Z', {COR_TABLE[x_err_id]})
    if z_err_id: state.run_gate('X', {COR_TABLE[z_err_id]})

    # Measure data qubits
    qc = pc.circuits.QuantumCircuit([{'measure Z': {0,1,2,3,4,5,6}}])
    msmt, _ = circ_runner.run(state, qc)
    msmt_bit_str = BitArray(bin='0000000') # msmt to bitstring
    for idx in list(*msmt.values()):
        msmt_bit_str[idx] = 1

    inv_msmt_bit_str = msmt_bit_str.uint ^ 0b1111111 # orthogonal state
    min_dist = min([hamming2(inv_msmt_bit_str, stab) for stab in stabs])
    cnt += 1

err2ticks = lambda errs: [list(err.values())[0]._ticks for _, err in errs.items()]
print("Correction to %s (|1_L>) due to errors:" % msmt_bit_str)
print("X-circuit:", err2ticks(e_x))
print("Z-circuit:", err2ticks(e_z))

print(cnt)

Correction to 0b0111111 (|1_L>) due to errors:
X-circuit: [[Tick({'Z': {9}})], [Tick({'Z': {7}}), Tick({'X': {5}})]]
Z-circuit: [[Tick({'X': {8}})], [Tick({'I': {2}}), Tick({'I': {8}})]]
1
